In [2]:
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt
import csv
import xlrd
import os
import os.path
from pathlib2 import Path
from decimal import *
%matplotlib inline

In [3]:
surface_areas = ['SA19-1_20ns', 'SA19-2_100ns', 'SA19-3_20ns',
                 'SA19.5_100ns', 'SA19.5-2_20ns', 'SA19.5-3_20ns',
                 'SA20_20ns', 'SA20-2_20ns', 'SA20-3_100ns',
                 'SA21_100ns', 'SA21-2_20ns', 'SA21-3_20ns', 
                 'SA22_20ns', 'SA22-2_20ns', 'SA22-3_100ns', 
                 'SA24_20ns', 'SA24-2_20ns', 'SA24-3_100ns']
n_surface_areas = ['SA19','SA19-2', 'SA19-3', 
                   'SA19.5','SA19.5-2', 'SA19.5-3',
                   'SA20','SA20-2', 'SA20-3',
                   'SA20.5', 'SA20.5-2', 'SA20.5-3',
                   'SA21','SA21-2', 'SA21-3',
                   'SA22','SA22-2', 'SA22-3',
                   'SA23','SA23-2', 'SA23-3',
                   'SA24', 'SA24-2', 'SA24-3']
n_filenames = ['step7.1', 'step7.2', 'step7.21', 'step7.3', 'step7.31', 'step7.32', 'step7.33', 'step7.34',
               'step7.35','step7.4', 'step7.41', 'step7.42', 'step7.43', 'step7.44', 'step7.45']
filedir = '/gpfs/amarolab/monolayers/MIX1_NaCl0.4_GMX/'
filedir1 = '/gpfs/amarolab/abbyflabby/MIX/MIX1234_NaCl0.4_NAMD/'
logfiles = []

for narea in n_surface_areas:
    logchunk = []
    for i in n_filenames:
        log = Path(filedir1+narea+'/charmm-gui/namd/'+i+'_production.log')
        if log.exists():
            logchunk.append(filedir1+narea+'/charmm-gui/namd/'+i+'_production.log')
    if len(logchunk)>0:
        logfiles.append(logchunk)
xvgfiles = [] #list of xvg filenames
pres = []
times = []
areas = []
trajfiles = []
#for narea in n_surface_areas:
#    logchunk = []
    
for area in surface_areas:
    xvgchunk = []
    trajchunk = []
    for i in range(1, 21):
        xvg = Path(filedir+area+'/step9_'+str(i)+'.xvg')
        traj = Path(filedir+area+'/step9_'+str(i)+'.trr')
        if xvg.exists():
            xvgchunk.append(filedir+area+'/step9_'+str(i)+'.xvg')
        if traj.exists():
            trajchunk.append(filedir+area+'/step9_'+str(i)+'.trr')
    #print(xvgchunk)
    for p, x in enumerate(xvgchunk):
        if p == 0:
            #arr = np.array(timesteps)
            timesteps = np.genfromtxt(x, dtype='float', skip_header=24)[:,0]
            #np.append(timesteps, new_ts)
            pressures = np.genfromtxt(x, dtype='float', skip_header=24)[:,1]/20
            #np.append(pressures, new_pr)
            #print(pressures)
        else:
            time = np.genfromtxt(x,dtype='float', skip_header=24)[:,0].tolist()
            #print(time)
            timesteps = np.append(timesteps, time)
      #      #print(time)
      #      #t = time.tolist()
      #      timesteps.append(time)
      #      print(timesteps)
            pr = np.genfromtxt(x, dtype='float', skip_header=24)[:,1]/20
            np.append(pressures, pr)
    if len(xvgchunk)>0:
        pres.append(70.5-np.array(pressures))
        times.append(timesteps)
        areas.append(area)
        xvgfiles.append(xvgchunk)
        trajfiles.append(trajchunk)
#print(len(xvgfiles))
#print(xvgfiles)
#print(len(times))
#print(times)
#print(len(pres))
#print(len(times))
#print(areas)

## Nematic Order parameter calculation

In [5]:
## need to create an array of arrays with the indices of EACH atom in each residue
##[[atom indices for res#1], [atom indices for res#2]] ...etc
nemData = []
count = 0
#surface_areas = ["19"]
for area in surface_areas:
    nemOrderIdx = []
    resID = []
    chunk = []
    #t_all = md.load(dcds[count], top=filedir1+"SA"+area+"/charmm-gui/step5_assembly.xplor_ext.psf")
    t_all = md.load(trajfiles[count], top=filedir+area+"/step9_1.gro")
    t_pdb = t_all[-1].save_pdb('PDB.pdb')
    lip = t_all.topology.select('resname LAUP or resname MYRP or resname PALP or resname STEP')
    t = t_all.atom_slice(lip)
    p = md.load_pdb('PDB.pdb')
    #p = md.load_pdb(filedir1+"/SA19_last_rm.pdb")
    resNum =p.n_residues 
    lines = open('PDB.pdb', "r")
    #lines = open(filedir1+"/SA19_last_rm.pdb", "r")
    for line in lines:
        if line[0] == "A":
            #print(line[6:11]) #atom index
            #print(line[23:26])##residue sequence number
            resIDCur = line[23:26]
            #print(resIDCur)
            resID.append(resIDCur)
            atomIdx = int(line[6:11].replace(" ", ""))
            #print(atomIdx)
            if len(resID) > 1:
                if len(resID) == resNum - 1:
                    chunk.append(atomIdx)
                    nemOrderIdx.append(chunk)
                #print(resID[-1])
                #print(resID[-2])
                #print('END')
                elif resID[-1] == resID[-2]: #if the last thing added to the resID guy is the same 
                #as the previous guy added, then there has been no change in the residue, so we should keep 
                #adding indices to chunk
                    #print('here')
                    chunk.append(atomIdx)
                    #print(chunk)
                #else, we need to append the chunk to the nematic order index and start a new chunk
                else:
                    nemOrderIdx.append(chunk)
                    chunk = []
                    chunk.append(atomIdx)
                #if resSeqCur != resSeq: 
                #    nemOrderIdx.append(chunk)
                #else:
                #    chunk.append(atomIdx)
            else:
                chunk.append(atomIdx)
    s = md.compute_nematic_order(t, indices = nemOrderIdx)
    nemData.append(s)
    count+=1

IndexError: index 18772 is out of bounds for axis 1 with size 18772